D'abord : se logguer à un compte gmail \
login : eleveenmi@gmail.com  (i=1,2,3,..) \
mwd : pwdeleveEnmi (i=1,2,3,..)

# I) Installations et imports

In [1]:
# Google sheet
!pip install --upgrade gspread gspread_dataframe

In [3]:
# Imports
from google.colab import auth
from google.auth import default
import gspread
from google.colab import drive
from google.colab import output
import os
import re
import shutil
import tarfile
from pathlib import Path
import random
from IPython.display import display, clear_output, Javascript
import ipywidgets as widgets
from base64 import b64encode
from datetime import datetime
import pandas as pd
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import re


# II) Authentification

In [2]:
# pour colab :
auth.authenticate_user()
# pour accéder aux fichiers google (google sheet)
creds, _ = default()  # or use service account
gc = gspread.authorize(creds)

# montage du drive :
drive.mount('/content/drive')

NameError: name 'auth' is not defined

# III) Configuration

In [ ]:
group=13
direction, starthour = group2direction_and_starthour[group]
image_dir = f'{group}_{direction}_{starthour}'
tar_and_zipped_image_dir = image_dir + '.tar.gz'
starting_image_idx = 3

# IV) Récupération des images :

In [ ]:
# 1. Define the source path (change SHARED_PATH accordingly)
shared_path = '/content/drive/MyDrive/' + tar_and_zipped_image_dir
dest_path = Path.home() / tar_and_zipped_image_dir

# Copy the file to your home directory
shutil.copy(shared_path, dest_path)

# 2. Extract and delete the archive
with tarfile.open(dest_path, 'r:gz') as tar:
    tar.extractall(path='.')  # extract to current working directory

# 3. Remove the tar.gz file
dest_path.unlink()

# V) Récupération d'une partie des labels :

In [ ]:
# Open by ID or URL
sheet_url = group2sheets[group]
sh = gc.open_by_url(sheet_url)
worksheet = sh.sheet1

# VI) Annotation

In [ ]:
# get dict_labels
dict_labels = build_initial_dict_from_images(image_dir)

# Complete it with data from google worksheet
complete_dict_from_worksheet(worksheet, dict_labels)

# get all subfolders :
# Root folder
root_folder = image_dir
# Collect subfolders
subfolders = sorted([
    os.path.join(root_folder, d) for d in os.listdir(root_folder)
    if os.path.isdir(os.path.join(root_folder, d))
])

# define button color dict :
color_map = assign_colors(dict_labels, direction)

# Inject once (put this near the beginning of your notebook)
display(widgets.HTML("""
<style>
.widelabel input {
    width: 100% !important;
    font-size: 16px;
    padding: 6px;
    display: block;
    margin: 0 auto;
}
</style>
"""))

# Extract timestamp from folder names (just first token)
timestamps = [os.path.basename(f).split('_')[0] for f in subfolders]

# Assign a unique color to each timestamp (simple hash function)
# color_map = {}
# for ts in set(timestamps):
#     random.seed(ts)
#     color_map[ts] = f"#{random.randint(50, 255):02x}{random.randint(50, 255):02x}{random.randint(50, 255):02x}"

# State
current_subfolder_index = None
current_image_index = starting_image_idx

# Widgets
button_box = widgets.VBox()
image_html = widgets.HTML()
meta_label = widgets.Label()
meta_label2 = widgets.Label()
meta_label3 = widgets.Label()
status_label = widgets.Label()
result_output = widgets.Output()


label_input = widgets.Text(
    placeholder='Write a label & press Enter',
    layout=widgets.Layout(height='40px', display='none')  # no width here!
)
label_input.add_class("widelabel")


# label_input.add_class("label-textbox")
label_input.add_class("large-placeholder")
# Container for image view
viewer_box = widgets.VBox()
layout_box = widgets.VBox()  # master view that swaps button list and image viewer

# Close button (top-right)
close_button = widgets.Button(
    description='Fermeture pop-up',
    layout=widgets.Layout(width='20%', height='30px', align_self='flex-end'),
    style={
        'button_color': '#444444',
        'font_weight': 'bold',
        'text_color': 'white'
    }
)


# === BUTTON LIST INITIALIZATION ===
folder_buttons = []

for idx, folder in enumerate(subfolders):
    folder_name = os.path.basename(folder)
    ts_raw = folder_name.split('_')[0]

    # Format the timestamp
    try:
        dt = datetime.strptime(ts_raw, "%Y%m%d%H%M")
        readable_dt = dt.strftime("%d/%m/%Y %H:%M")
    except Exception:
        readable_dt = "??/??/???? ??:??"

    # Get third image filename
    try:
        image_files = sorted([
            f for f in os.listdir(folder)
            # if f.lower().endswith(('.jpg', '.jpeg', '.png'))
        ])
        third_image_name = image_files[starting_image_idx] if len(image_files) > starting_image_idx else None
    except Exception:
        third_image_name = None

    # Get interpreted label
    # if third_image_name:
    # try:
    label_text = get_interpreted_labels(dict_labels, third_image_name, correspondance)
    # except Exception as e:
    #     label_text = f"Error: {e}"
    # else:
    #     label_text = "(No third image)"

    # Build display label
    full_label = f"{readable_dt}\n{label_text}"

    # Create button
    btn = widgets.Button(
        description=full_label,
        layout=widgets.Layout(width='60%', height='23px'),
        style={
            'button_color': color_map.get(ts_raw, '#dddddd'),
            'font_weight': 'bold',
            'text_color': 'black'
        }
    )
    btn._idx = idx  # attach index for handler

    def on_click(b):
        show_image_view(b._idx)

    btn.on_click(on_click)
    folder_buttons.append(btn)


# button_box.children = [widgets.HBox(folder_buttons[i:i+5]) for i in range(0, len(folder_buttons), 5)]
button_box.children = [widgets.VBox(folder_buttons)]

# scrollable_vbox = widgets.VBox(folder_buttons)

# scrollable_container = widgets.Box(
#     [scrollable_vbox],
#     layout=widgets.Layout(
#         max_height='500px',      # set vertical limit
#         overflow='auto',         # enable scrolling
#         border='1px solid lightgray',
#         padding='5px',
#         width='100%'             # optional: full width
#     )
# )

# button_box.children = [scrollable_container]

# === VIEWER BOX LAYOUT ===
label_input.on_submit(on_label_submit)
close_button.on_click(return_to_buttons)
viewer_box.children = [meta_label, meta_label2, meta_label3,  image_html, status_label, label_input, result_output, widgets.HBox([close_button])]

# === CALLBACK REGISTRATION ===
output.register_callback('notebook.on_left', on_left)
output.register_callback('notebook.on_right', on_right)
output.register_callback('notebook.on_up', on_up)
output.register_callback('notebook.on_down', on_down)

# === DISPLAY INITIAL VIEW ===
layout_box.children = [button_box]
display(layout_box)

# A) Récupération des premiers labels

In [ ]:
# Récupération et transformation des labels déjà portés
# Upload ?

from google.colab import files
uploaded = files.upload()


In [ ]:
import os
import tarfile
from pathlib import Path
import re

# Step 1: Detar the file
tar_path = 'classification_images.tar'  # replace with your actual tar file
extract_path = Path('extracted')
with tarfile.open(tar_path, 'r:*') as tar:
    tar.extractall(extract_path)

# Step 1.1: Find the actual root folder (the one containing subfolders)
extracted_items = list(extract_path.iterdir())
if len(extracted_items) != 1 or not extracted_items[0].is_dir():
    raise RuntimeError("Expected exactly one root folder in the tar file.")
true_root = extracted_items[0]

# Step 2: Init dict_labels
dict_labels = {}

# Strict regex: only match S_YYYY-MM-DD_HH_MM_SS.jpg
strict_pattern = re.compile(r'^S_(\d{4})-(\d{2})-(\d{2})_(\d{2})_(\d{2})_(\d{2})\.jpg$')

# Step 3: Browse subfolders under the true root
for subfolder in true_root.iterdir():
    if not subfolder.is_dir():
        continue

    label = subfolder.name.split('-')[0].split('_')[0]

    for item in subfolder.iterdir():
        if item.is_file():
            fname = item.name
            if fname == "S_2025-04-16_19_00_00.jpg":
                continue  # Explicitly skip this one
            if not strict_pattern.match(fname):
                continue  # Skip anything that doesn't match the strict pattern

            # Reformat: from S_YYYY-MM-DD_HH_MM_SS.jpg to S_YYYYMMDDHHMM.jpg
            parts = strict_pattern.match(fname).groups()
            new_name = f"S_{parts[0]}{parts[1]}{parts[2]}{parts[3]}{parts[4]}.jpg"

            if new_name in dict_labels:
                print(f"[DUPLICATE] {new_name} already labeled.")
            dict_labels[new_name] = label

        elif item.is_dir():
            for subitem in item.iterdir():
                if subitem.is_file():
                    fname = subitem.name
                    if fname == "S_2025-04-16_19_00_00.jpg":
                        continue
                    if not strict_pattern.match(fname):
                        continue

                    parts = strict_pattern.match(fname).groups()
                    new_name = f"S_{parts[0]}{parts[1]}{parts[2]}{parts[3]}{parts[4]}.jpg"

                    if new_name in dict_labels:
                        print(f"[DUPLICATE] {new_name} already labeled.")
                    dict_labels[new_name] = label

                elif subitem.is_dir():
                    print(f"[WARNING] Found subsubsubfolder in: {subitem.relative_to(true_root)}")

print("Label dictionary created with", len(dict_labels), "entries.")


In [ ]:
# ✅ Install dependencies (Colab only)
!pip install --upgrade gspread gspread_dataframe google-auth

# ✅ Imports and auth
import gspread
from gspread_dataframe import set_with_dataframe
import pandas as pd
import tarfile
from pathlib import Path
import re

from google.colab import auth
auth.authenticate_user()

import google.auth
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

In [ ]:
! ls extracted/


classification_images


In [ ]:
# ✅ Convert to DataFrame
df = pd.DataFrame(list(dict_labels.items()), columns=["Image", "Classe"])

# ✅ Create and fill Google Sheet
sheet_title = "group0_group1_labels"
sh = gc.create(sheet_title)
worksheet = sh.sheet1
set_with_dataframe(worksheet, df)




In [ ]:
# ✅ Share with eleveenm1@gmail.com to eleveenm12@gmail.com
for k in range(1, 13):
    email = f"eleveenm{k}@gmail.com"
    try:
        sh.share(email, perm_type='user', role='writer')
        print(f"✅ Shared with {email}")
    except Exception as e:
        print(f"❌ Failed to share with {email}: {e}")


# B) Création des google sheets

In [ ]:
# ✅ Install dependencies (Colab only)
!pip install --upgrade gspread gspread_dataframe google-auth

# ✅ Imports and auth
import gspread
from gspread_dataframe import set_with_dataframe
import pandas as pd
import tarfile
from pathlib import Path
import re

from google.colab import auth
auth.authenticate_user()

import google.auth
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

In [ ]:
# ✅ Create and fill Google Sheet



for k in range(1, 13):
    sheet_title = f"group{k}"
    sh = gc.create(sheet_title)
    print(f"\n✅ Sheet created: https://docs.google.com/spreadsheets/d/{sh.id}")
    email = f"eleveenm{k}@gmail.com"

    try:
        sh.share(email, perm_type='user', role='writer')
        print(f"✅ Shared with {email}")
    except Exception as e:
        print(f"❌ Failed to share with {email}: {e}")


✅ Sheet created: https://docs.google.com/spreadsheets/d/1JR3T7l6TrQdxKx1Vyxc-XM8q736PnYyGq5iJ81PUs1Y
✅ Shared with eleveenm1@gmail.com

✅ Sheet created: https://docs.google.com/spreadsheets/d/1Npi7H-Y-P60QBODOL8Q5ArQHr5dIn9mOuYv67LLHcNE
✅ Shared with eleveenm2@gmail.com

✅ Sheet created: https://docs.google.com/spreadsheets/d/1I_ZwJrPt1AO_tQJSX2JBreFFhAlsF7kITH9p_1gDhks
✅ Shared with eleveenm3@gmail.com

✅ Sheet created: https://docs.google.com/spreadsheets/d/1e1LnjbLKoZ_JxYfGLu1nFIQWyioNt4AV_Y9C1CUqiWI
✅ Shared with eleveenm4@gmail.com

✅ Sheet created: https://docs.google.com/spreadsheets/d/1unaaeXdGVdC_1JMMVSBUgxc6vdrJzE16lTiP6OBEl04
✅ Shared with eleveenm5@gmail.com

✅ Sheet created: https://docs.google.com/spreadsheets/d/1odXOBEiNbFQ48NALzTEmCs7LDlBHWARdsivL8rO47fw
✅ Shared with eleveenm6@gmail.com

✅ Sheet created: https://docs.google.com/spreadsheets/d/1v7gJmdK1AqMelBRaksIcrIN_mDjobH8wxMxfLbiKlS8
✅ Shared with eleveenm7@gmail.com

✅ Sheet created: https://docs.google.com/spread

In [ ]:
group2sheets = {0: 'https://docs.google.com/spreadsheets/d/1sjvrkLMEY0YXAyXU9Ad8i49J29jCmKkJb85T2pZ2CS8',
                1: 'https://docs.google.com/spreadsheets/d/1sjvrkLMEY0YXAyXU9Ad8i49J29jCmKkJb85T2pZ2CS8',
                2: 'https://docs.google.com/spreadsheets/d/1Npi7H-Y-P60QBODOL8Q5ArQHr5dIn9mOuYv67LLHcNE',
                3: 'https://docs.google.com/spreadsheets/d/1I_ZwJrPt1AO_tQJSX2JBreFFhAlsF7kITH9p_1gDhks',
                4: 'https://docs.google.com/spreadsheets/d/1e1LnjbLKoZ_JxYfGLu1nFIQWyioNt4AV_Y9C1CUqiWI',
                5: 'https://docs.google.com/spreadsheets/d/1unaaeXdGVdC_1JMMVSBUgxc6vdrJzE16lTiP6OBEl04',
                6: 'https://docs.google.com/spreadsheets/d/1odXOBEiNbFQ48NALzTEmCs7LDlBHWARdsivL8rO47fw',
                7: 'https://docs.google.com/spreadsheets/d/1v7gJmdK1AqMelBRaksIcrIN_mDjobH8wxMxfLbiKlS8',
                8: 'https://docs.google.com/spreadsheets/d/1chlVgydwqpQYq44ynEWX_I0QVtWsUaYfwqVSptTD-dA',
                9: 'https://docs.google.com/spreadsheets/d/1x295asse_gh31-EzXQYFUSIkpbLe-hvYwWAxyr1c_ag',
                10: 'https://docs.google.com/spreadsheets/d/1QKZuVjBd1SOmAZvg1zA4vnzLnh8DXVzDouwA941WB2I',
                11: 'https://docs.google.com/spreadsheets/d/1oNurDfrYvCr0hnawUz432Dyh8Mkunk820GiDoa1Sz_E',
                12: 'https://docs.google.com/spreadsheets/d/1EgmnmcX8jMXZmz6s_-oW473FnuT3HgPjlSnqCcJTrP4',
                13 : 'https://docs.google.com/spreadsheets/d/1KT4RXpX3FkR2APlGXhD54euEpPWHbFZKrxoWdFzeqog'}